## [作業重點]
了解如何使用 Sklearn 中的 hyper-parameter search 找出最佳的超參數

什麼是模型超參數？

模型超參數是模型外部的配置，其值不能從數據估計得到。

具體特徵有：

- 模型超參數常應用於估計模型參數的過程中。

- 模型超參數通常由實踐者直接指定。

- 模型超參數通常可以使用啟發式方法來設置。

- 模型超參數通常根據給定的預測建模問題而調整。

怎樣得到它的最優值：對於給定的問題，我們無法知道模型超參數的最優值。但我們可以使用經驗法則來探尋其最優值，或複製用於其他問題的值，也可以通過反覆試驗的方法。

模型超參數的一些例子包括：

- 訓練神經網絡的學習速率。

- 支持向量機的C和sigma超參數。

- k鄰域中的k。



原文網址：https://kknews.cc/tech/yy58k3b.html

### 作業
請使用不同的資料集，並使用 hyper-parameter search 的方式，看能不能找出最佳的超參數組合

In [1]:
from sklearn import datasets, metrics
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor

In [2]:
# 讀取手寫辨識資料集
boston = datasets.load_boston()
# 切分訓練集/測試集
x_train, x_test, y_train, y_test = train_test_split(boston.data, boston.target, test_size=0.25, random_state=42)

# 建立模型
clf = GradientBoostingRegressor(random_state=7)

In [3]:
# 先看看使用預設參數得到的結果，約為 8.91 MSE
clf.fit(x_train, y_train)
y_pred = clf.predict(x_test)
print(metrics.mean_squared_error(y_test, y_pred))

8.913775994322062


In [4]:
# 設定要訓練的超參數組合
n_estimators = [100, 200, 300]
max_depth = [1, 3, 5]
param_grid = dict(n_estimators=n_estimators, max_depth=max_depth)

## 建立搜尋物件，放入模型及參數組合字典 (n_jobs=-1 會使用全部 cpu 平行運算)
grid_search = GridSearchCV(clf, param_grid, scoring="neg_mean_squared_error", n_jobs=-1, verbose=1)

# 開始搜尋最佳參數
grid_result = grid_search.fit(x_train, y_train)

# 預設會跑 3-fold cross-validadtion，總共 9 種參數組合，總共要 train 27 次模型

Fitting 5 folds for each of 9 candidates, totalling 45 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  45 out of  45 | elapsed:    3.4s finished


In [5]:
# 印出最佳結果與最佳參數
print("Best Accuracy: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Best Accuracy: -10.843075 using {'max_depth': 3, 'n_estimators': 100}


In [6]:
grid_result.best_params_

{'max_depth': 3, 'n_estimators': 100}

In [7]:
# 使用最佳參數重新建立模型
clf_bestparam = GradientBoostingRegressor(max_depth=grid_result.best_params_['max_depth'],
                                           n_estimators=grid_result.best_params_['n_estimators'])

# 訓練模型
clf_bestparam.fit(x_train, y_train)

# 預測測試集
y_pred = clf_bestparam.predict(x_test)

In [8]:
# 調整參數後約可降至 8.30 的 MSE
print(metrics.mean_squared_error(y_test, y_pred))

8.603309182808905
